In [1]:
import pandas as pd
import numpy as np

# Load the data
data_path = 'car_filled.xlsx'
car_data = pd.read_excel(data_path)

# Data preprocessing function
def preprocess_data(df):
    """
    Preprocess the data, including handling columns with multiple values and one-hot encoding fuel types.
    """
    # Handle columns with multiple values (e.g., number of seats, starting MSRP), calculate the average
    for col in df.columns:
        if 'Number of seats' in col or 'Starting MSRP' in col or 'Safety score' in col:
            # Convert the values in the column to strings, then split by '/' and calculate the average
            df[col] = df[col].apply(lambda x: np.mean([float(i) for i in str(x).split('/') if i.isdigit()]))

    # One-hot encode fuel types
    fuel_cols = [col for col in df.columns if 'fuel' in col]
    for col in fuel_cols:
        # Get all possible fuel types
        all_fuels = set('/'.join(df[col].astype(str).tolist()).split('/'))
        for fuel in all_fuels:
            # Create a new column for each fuel type, set to 1 if the current row contains that fuel type, else 0
            df[f'{col}_{fuel.strip()}'] = df[col].apply(lambda x: 1 if fuel.strip() in str(x) else 0)
        # Drop the original fuel type column
        df.drop(col, axis=1, inplace=True)

    return df

# Re-run the preprocessing function
preprocessed_data = preprocess_data(car_data.copy())

# Display the first few rows of the preprocessed data again
preprocessed_data.head()


,State,Car1-Name,Car1-Number of seats,Car1-Starting MSRP,Car1-Safety score,Car2-Name,Car2-Number of seats,Car2-Starting MSRP,Car2-Safety score,Car3-Name,...,Car3-fuel_CNG,Car3-fuel_gasoline,Car4-fuel_diesel,Car4-fuel_electricity,Car4-fuel_CNG,Car4-fuel_gasoline,Car5-fuel_diesel,Car5-fuel_electricity,Car5-fuel_CNG,Car5-fuel_gasoline
0,Alabama,Chevrolet Silverado,6.0,36800.0,5.0,Ford F-Series,6.0,39895.0,5.0,Toyota Camry,...,0,1,0,0,0,1,1,0,0,0
1,Alaska,Ram 1500/2500/3500,5.0,45250.0,4.0,Chevrolet Silverado,6.0,36800.0,5.0,Ford F-Series,...,0,1,1,0,0,0,0,0,0,1
2,Arizona,Chevrolet Silverado,6.0,36800.0,5.0,Ram 1500/2500/3500,5.0,45250.0,4.0,Ford F-Series,...,0,1,0,0,0,1,0,0,0,1
3,Arkansas,GMC Sierra,3.0,39695.0,5.0,Ford F-Series,6.0,39895.0,5.0,Ram 1500/2500/3500,...,1,0,1,0,0,0,0,0,0,1
4,California,Tesla Model Y,5.0,37490.0,5.0,Tesla Model 3,5.0,38990.0,5.0,Toyota RAV4,...,0,1,0,0,0,1,0,0,0,1


In [9]:
def calculate_weighted_average(df):
    weights = {1: 5, 2: 4, 3: 3, 4: 2, 5: 1}
    results = []
    
    for state in df['State'].unique():
        state_data = df[df['State'] == state]
        weighted_sums = {}
        total_weights = 0
        
        for i in range(1, 6):
            for feature in ['Number of seats', 'Starting MSRP', 'Safety score']:
                col_name = f'Car{i}-{feature}'
                if col_name in state_data.columns:
                    if feature not in weighted_sums:
                        weighted_sums[feature] = state_data[col_name].values[0] * weights[i]
                    else:
                        weighted_sums[feature] += state_data[col_name].values[0] * weights[i]
            
            fuel_cols = [col for col in df.columns if col.startswith(f'Car{i}-fuel')]
            for col in fuel_cols:
                if col.split('_')[-1] not in weighted_sums:
                    weighted_sums[col.split('_')[-1]] = state_data[col].values[0] * weights[i]
                else:
                    weighted_sums[col.split('_')[-1]] += state_data[col].values[0] * weights[i]
            
            total_weights += weights[i]
        
        for key in weighted_sums:
            weighted_sums[key] /= total_weights
        
        fuel_types = {key: value for key, value in weighted_sums.items() if key in ['gasoline', 'diesel', 'electricity', 'CNG', 'nan']}
        max_fuel_type = max(fuel_types, key=fuel_types.get)
        
        result = {
            'State': state,
            'Average Number of Seats': weighted_sums['Number of seats'],
            'Average Starting MSRP': weighted_sums['Starting MSRP'],
            'Average Safety Score': weighted_sums['Safety score'],
            'Most Common Fuel Type': max_fuel_type
        }
        results.append(result)
    
    return pd.DataFrame(results)

weighted_average_results = calculate_weighted_average(preprocessed_data)

weighted_average_results.head()

,State,Average Number of Seats,Average Starting MSRP,Average Safety Score,Most Common Fuel Type
0,Alabama,5.466667,34659.000000,5.000000,gasoline
1,Alaska,5.200000,40080.000000,4.666667,diesel
2,Arizona,5.400000,38424.000000,4.600000,gasoline
3,Arkansas,4.733333,39738.666667,4.800000,diesel
4,California,5.000000,33621.666667,5.000000,electricity


In [10]:
# Adjusting the weighted average calculation function without specifically boosting non-gasoline vehicles
def calculate_weighted_average_exponential_no_boost_en(df):
    weights = {i: 2 ** (5 - i) for i in range(1, 6)}  # Exponential weight distribution
    results = []
    
    for state in df['State'].unique():
        state_data = df[df['State'] == state]
        weighted_sums = {}
        total_weights = 0
        
        for i in range(1, 6):
            for feature in ['Number of seats', 'Starting MSRP', 'Safety score']:
                col_name = f'Car{i}-{feature}'
                if col_name in state_data.columns:
                    # Add or update the weighted sum for each feature
                    if feature not in weighted_sums:
                        weighted_sums[feature] = state_data[col_name].values[0] * weights[i]
                    else:
                        weighted_sums[feature] += state_data[col_name].values[0] * weights[i]
            
            # Apply normal weights for fuel types
            fuel_cols = [col for col in df.columns if col.startswith(f'Car{i}-fuel')]
            for col in fuel_cols:
                fuel_type = col.split('_')[-1]
                # Add or update the weighted sum for each fuel type
                if fuel_type not in weighted_sums:
                    weighted_sums[fuel_type] = state_data[col].values[0] * weights[i]
                else:
                    weighted_sums[fuel_type] += state_data[col].values[0] * weights[i]
            
            total_weights += weights[i]
        
        # Calculate the weighted average for each feature and fuel type
        for key in weighted_sums:
            weighted_sums[key] /= total_weights
        
        # Determine the most common fuel type based on the highest weighted average
        fuel_types = {key: value for key, value in weighted_sums.items() if key in ['gasoline', 'diesel', 'electricity', 'CNG', 'nan']}
        max_fuel_type = max(fuel_types, key=fuel_types.get)
        
        # Compile the result for the current state
        result = {
            'State': state,
            'Average Number of Seats': weighted_sums['Number of seats'],
            'Average Starting MSRP': weighted_sums['Starting MSRP'],
            'Average Safety Score': weighted_sums['Safety score'],
            'Most Common Fuel Type': max_fuel_type
        }
        results.append(result)
    
    return pd.DataFrame(results)

# Recalculate the weighted averages without specifically boosting non-gasoline vehicles, using English comments
weighted_average_results_exponential_no_boost_en = calculate_weighted_average_exponential_no_boost_en(preprocessed_data)


# Output the results to a new Excel file
output_path = 'weighted_average_results.xlsx'
weighted_average_results_exponential_no_boost_en.to_excel(output_path, index=False)

# Display the results
weighted_average_results_exponential_no_boost_en.head()

,State,Average Number of Seats,Average Starting MSRP,Average Safety Score,Most Common Fuel Type
0,Alabama,5.709677,35828.548387,5.000000,diesel
1,Alaska,5.258065,41485.322581,4.483871,CNG
2,Arizona,5.580645,38775.967742,4.677419,diesel
3,Arkansas,4.290323,39921.129032,4.870968,diesel
4,California,5.000000,35527.419355,5.000000,electricity
